In [ ]:
# Establish
# With inspiration from https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/fall2021/extra_topics/twitter_v2_example.ipynb
import os
import tweepy
import pandas as pd
import json
import time
import csv

In [ ]:
FETCH_NEW_TWEETS = False

with open(os.path.join('..', 'keys', 'keys.json')) as f:
    twitter_keys = json.load(f)

client = tweepy.Client(bearer_token=twitter_keys['bearer_token'], wait_on_rate_limit=True)

In [ ]:
from datetime import datetime, timezone

In [ ]:
def write_csv(data, name='name', new=False):
    with open(os.path.join('data', f'{name}.csv'), 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        if new:  # new file, writing header
            writer.writerow(data)
        else:
            writer.writerows(data)

In [ ]:
start_time = '2014-03-16T00:00:00Z'
end_time = '2014-03-23T00:00:00Z'

MAX_TWEETS = 1000
MAX_TWEETS_PER_FILE = 100
tweets_in_file = 0
total_fetched_tweets = 0
max_tweets_per_page = 100
query = 'ukraine russia lang:en'

start_dt = datetime.strptime(start_time,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc)
end_dt = datetime.strptime(end_time,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc)
total_days = (end_dt - start_dt).days

filename = ''  # gets set iteratively for each new file

fields = ['id','text','author_id','created_at','geo']

for response in tweepy.Paginator(client.search_all_tweets,
            query = query,
            tweet_fields = fields,
            start_time = start_time,
            end_time = end_time,
            max_results=max_tweets_per_page               
):  
    max_date = max([tweet.created_at for tweet in response.data])
    days_fetched = (end_dt - max_date).days
    if days_fetched % 1 == 0:
        print(f'Fetched {days_fetched}/{total_days}, ({100*days_fetched/total_days}%)')

    if tweets_in_file > MAX_TWEETS_PER_FILE or tweets_in_file == 0:
        tweets_in_file = 0

        filename = f'{max_date.strftime("%Y%m%d%H%M%S")}'
        print(f'new filename: {filename}')

        write_csv(fields, filename, new=True)  # write headers to file
        print('Created new file.')
    

    tweet_list = [[tweet.id, tweet.text.replace('\n',' '), tweet.author_id, tweet.created_at, tweet.geo] for tweet in response.data]
    write_csv(tweet_list, filename)
    print('Wrote response to file.')
    time.sleep(1)
    tweets_in_file += response.meta['result_count']
    total_fetched_tweets += response.meta['result_count']

    if total_fetched_tweets > MAX_TWEETS:
        break


In [ ]:

f = '/Users/olaaflo/Desktop/Master Year 1/Data-Intensive-Sys/Project/Twitter_Sentiment/data/20140322224712.csv'
with open(f, 'r') as file:
    reader = csv.reader(file)
    for line in reader:
        print(line)